
# Работа с HTTP и публичными API (Python + requests)

**Автогенерация:** 2025-11-07 15:06:15  
Инструменты: Jupyter/Colab/VS Code.  
Пакеты: `requests` (в Colab уже есть).

> В ноутбуке задания 1–4 с примерами. Просто запускайте ячейки сверху вниз.  
> Для OpenWeather нужен API‑ключ — в ноутбуке показано, куда его вставить.


In [8]:

# ⬇️ Установка (на всякий случай) и импорт
# В Colab обычно не требуется, но в локальной среде можно раскомментировать строку ниже
# !pip install requests python-dotenv

import os
import sys
import json
import requests
from typing import Optional

from dotenv import load_dotenv
load_dotenv()

print("Python:", sys.version.split()[0])
print("requests:", requests.__version__)


Python: 3.13.3
requests: 2.32.5


In [9]:

def pretty(obj):
    print(json.dumps(obj, ensure_ascii=False, indent=2))



## Задание 1. Получение данных из публичного API

**API:** `https://jsonplaceholder.typicode.com/posts`  
**Задача:** отправить GET-запрос к `/posts` и вывести заголовки и тела первых 5 постов.


In [10]:

BASE = "https://jsonplaceholder.typicode.com"

resp = requests.get(f"{BASE}/posts", timeout=20)
resp.raise_for_status()  # Если код не 2xx — бросит исключение

posts = resp.json()[:5]
for i, p in enumerate(posts, 1):
    print(f"#{i}. {p['title']}")
    print(p['body'])
    print("-" * 60)


#1. sunt aut facere repellat provident occaecati excepturi optio reprehenderit
quia et suscipit
suscipit recusandae consequuntur expedita et cum
reprehenderit molestiae ut ut quas totam
nostrum rerum est autem sunt rem eveniet architecto
------------------------------------------------------------
#2. qui est esse
est rerum tempore vitae
sequi sint nihil reprehenderit dolor beatae ea dolores neque
fugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis
qui aperiam non debitis possimus qui neque nisi nulla
------------------------------------------------------------
#3. ea molestias quasi exercitationem repellat qui ipsa sit aut
et iusto sed quo iure
voluptatem occaecati omnis eligendi aut ad
voluptatem doloribus vel accusantium quis pariatur
molestiae porro eius odio et labore et velit aut
------------------------------------------------------------
#4. eum et est occaecati
ullam et saepe reiciendis voluptatem adipisci
sit amet autem assumenda provident rerum culpa
quis hic


## Задание 2. Работа с параметрами запроса (OpenWeather)

**API:** <https://openweathermap.org/current>  
**Что нужно:** ваш API‑ключ. Получить можно бесплатно после регистрации на openweathermap.org.

**Ввод:** название города.  
**Вывод:** текущая температура (°C) и описание погоды.


In [12]:

# 🔑 Вариант А: вставьте ключ прямо сюда (не для публичных репозиториев!)
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY") or ""  # например: "abc123..."

# 🔑 Вариант Б (рекомендуется локально): экспортируйте переменную окружения
# В bash: export OPENWEATHER_API_KEY="ваш_ключ"
# В PowerShell: $Env:OPENWEATHER_API_KEY="ваш_ключ"

city = input("Введите город (например, Moscow): ").strip() or "Moscow"

if not OPENWEATHER_API_KEY:
    raise SystemExit("Нет API‑ключа. Установите переменную OPENWEATHER_API_KEY или вставьте ключ в переменную OPENWEATHER_API_KEY выше.")

url = "https://api.openweathermap.org/data/2.5/weather"
params = {
    "q": city,
    "appid": OPENWEATHER_API_KEY,
    "units": "metric",     # метры/сек и градусы Цельсия
    "lang": "ru",          # описание на русском
}
r = requests.get(url, params=params, timeout=20)

if r.status_code == 200:
    data = r.json()
    temp = data["main"]["temp"]
    desc = data["weather"][0]["description"]
    print(f"Сейчас в {city}: {temp:.1f}°C, {desc}.")
else:
    print(f"Ошибка запроса: {r.status_code}")
    try:
        pretty(r.json())
    except Exception:
        print(r.text[:500])


Сейчас в Yakutsk: -22.0°C, ясно.



## Задание 3. Создание и обработка POST‑запросов

**API:** `https://jsonplaceholder.typicode.com/posts`  
**Задача:** отправить POST для создания нового поста и вывести `id` и содержимое.
> JSONPlaceholder не создаёт реальные записи, но возвращает валидный ответ, которого достаточно для учебной задачи.


In [13]:

payload = {
    "title": "Учебный пост",
    "body": "Это тело поста, отправленного через POST.",
    "userId": 42,
}

r = requests.post(f"{BASE}/posts", json=payload, timeout=20)
print("Статус:", r.status_code)
data = r.json()
pretty(data)

post_id = data.get("id")
print("ID нового поста:", post_id)


Статус: 201
{
  "title": "Учебный пост",
  "body": "Это тело поста, отправленного через POST.",
  "userId": 42,
  "id": 101
}
ID нового поста: 101



## Задание 4. Обработка ошибок и разные коды состояния

Расширим код так, чтобы:
- различать **200/201** и обрабатывать тело ответа;
- ловить **400/404** и печатать осмысленное сообщение;
- аккуратно печатать тело ошибки, если сервер его вернул.


In [14]:

def safe_request(method: str, url: str, **kwargs) -> Optional[requests.Response]:
    try:
        resp = requests.request(method=method.upper(), url=url, timeout=20, **kwargs)
    except requests.RequestException as e:
        print("⚠️ Сетевая ошибка:", e)
        return None

    code = resp.status_code
    if code in (200, 201):
        print(f"✅ Успех {code}")
        return resp
    elif code == 400:
        print("❌ 400 Bad Request — проверьте параметры.")
    elif code == 401:
        print("❌ 401 Unauthorized — нужен токен/ключ доступа.")
    elif code == 403:
        print("❌ 403 Forbidden — доступ запрещён.")
    elif code == 404:
        print("❌ 404 Not Found — ресурс не найден.")
    elif code == 429:
        print("❌ 429 Too Many Requests — превысили лимит запросов.")
    elif 500 <= code < 600:
        print(f"❌ {code} Server Error — проблемы на стороне сервера.")
    else:
        print(f"ℹ️ Неожиданный код: {code}")

    # Показать тело ошибки, если есть
    try:
        body = resp.json()
    except Exception:
        body = resp.text[:500]
    print("Ответ сервера:")
    pretty(body)
    return None

# Пример 4.1: успешный GET
resp_ok = safe_request("GET", f"{BASE}/posts/1")
if resp_ok is not None:
    pretty(resp_ok.json())

# Пример 4.2: спровоцируем 404
resp_fail = safe_request("GET", f"{BASE}/posts/999999999")


✅ Успех 200
{
  "userId": 1,
  "id": 1,
  "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit",
  "body": "quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto"
}
❌ 404 Not Found — ресурс не найден.
Ответ сервера:
{}
